In [12]:
import os

In [13]:
def fileWalker(path):
    fileArray = []
    for roots, dirs, files in os.walk(path):
        for fn in files:
            eachpath = str(roots + '/' + fn)
            fileArray.append(eachpath)
    # print(fileArray)
    return fileArray

In [14]:
# fileWalker('/Users/zw/Desktop/Data Learning/Machine Learning Basics/Spam Filter')

In [15]:
def readText(path, encoding):
    with open(path, 'r', encoding=encoding) as f:
        lines = f.readlines()
    # print(lines)
    return lines

In [16]:
# readText('/Users/zw/Desktop/Data Learning/Machine Learning Basics/Spam Filter/ham/1.txt',
#          encoding='utf-8')

In [17]:
def email_parser(email_path):
    punctuations = """,.<>()*&^%$#@!'";~`[]{}|、\\/~+_-=?"""
    content_list = readText(email_path, 'utf-8')
    content = (' '.join(content_list)).replace('\r\n', ' ').replace('\t', ' ')  # LF, CR, HT
    clean_word = []
    for punctuation in punctuations:
        content = (' '.join(content.split(punctuation))).replace('  ', ' ')
        clean_word = [word.lower for word in content.split(' ') if len(word) > 2]
    # print(clean_word)
    return clean_word

In [18]:
# email_parser('/Users/zw/Desktop/Data Learning/Machine Learning Basics/Spam Filter/ham/1.txt')

In [19]:
def get_word(email_dir):
    word_list = []
    word_set = []
    email_paths = fileWalker(email_dir)
    for each_email_path in email_paths:
        clean_word = email_parser(each_email_path)
        word_list.append(clean_word)
        word_set.extend(clean_word)
    # print(word_list)
    # print(set(word_set))
    # print(type(word_set))
    return word_list, set(word_set)

In [20]:
# get_word('/Users/zw/Desktop/Data Learning/Machine Learning Basics/Spam Filter/ham/')

In [21]:
def count_word_prob(email_list, union_set):
    word_prob = {}
    for word in union_set:
        counter = 0
        for email in email_list:
            if word in email:
                counter += 1
            else:
                continue
        # prob = 0.0
        if counter != 0:
            prob = counter/len(email_list)
        else:
            prob = 0.01
        word_prob[word] = prob
    return word_prob

In [22]:
def myfilter(ham_word_pro, spam_word_pro, test_file):
    test_paths = fileWalker(test_file)
    for test_path in test_paths:
        # email_spam_prob = 0.0
        spam_prob = 0.5
        ham_prob = 0.5
        file_name = test_path.split('/')[-1]
        prob_dict = {}
        words = set(email_parser(test_path))
        for word in words:
            # Psw = 0.0
            if word not in spam_word_pro:
                Psw = 0.4
            else:
                Pws = spam_word_pro[word]
                Pwh = ham_word_pro[word]
                Psw = spam_prob*(Pws/(Pwh*ham_prob+Pws*spam_prob))
            prob_dict[word] = Psw
        numerator = 1
        denominator_h = 1
        for k, v in prob_dict.items():
            numerator *= v
            denominator_h *= (1-v)
        email_spam_prob = round(numerator/(numerator+denominator_h), 4)
        if email_spam_prob > 0.5:
            print(file_name, 'Spam', email_spam_prob)
        else:
            print(file_name, 'Ham', email_spam_prob)

In [23]:
def main():
    ham_file = './ham'
    spam_file = './spam'
    test_file = './test'
    ham_list, ham_set = get_word(ham_file)
    spam_list, spam_set = get_word(spam_file)
    union_set = ham_set | ham_set
    ham_word_pro = count_word_prob(ham_list, union_set)
    spam_word_pro = count_word_prob(spam_list, union_set)
    myfilter(ham_word_pro, spam_word_pro, test_file)

In [24]:
main()

10.txt Ham 0.0
9.txt Ham 0.0001
8.txt Ham 0.0
5.txt Ham 0.0005
4.txt Ham 0.0
6.txt Ham 0.0
7.txt Ham 0.0001
3.txt Ham 0.0
2.txt Ham 0.0
1.txt Ham 0.0002
